This is a notebook for caluclating the resistivity profile at SNMR sites using the AEM survey nearest point

Neil Symingotn

neil.symington@ga.gov.au


In [62]:
import pandas as pd
import numpy as np
import netCDF4
from hydrogeol_utils import spatial_functions
from geophys_utils import NetCDFPointUtils
import os

In [63]:
infile =r"E:\SSC\NMR\SNMR\SSC_SNMR_location_metadata.csv"

df_sites = pd.read_csv(infile)

In [64]:
df_sites

,Field_ID,Site_Code,X,Y,bo_dec_angle_deg,bo_inc_angle_deg,date,loop_width,coil_type,geometry
0,AS_GMR_02,NaN,380595.1,7361869.7,-55.525,4.614,NaN,110,1,NaN
1,AS_GMR_07,NaN,383653.7,7363414.8,-55.507,4.621,NaN,110,1,NaN
2,AS_GMR_08,NaN,391700.5,7358847.9,-55.551,4.649,NaN,110,1,NaN
3,AS_GMR_09,NaN,393272.9,7352590.9,-55.619,4.660,NaN,110,1,NaN
4,AS_GMR_26,NaN,390924.3,7362787.4,-55.509,4.643,NaN,110,1,NaN
5,AS_GMR_30,NaN,397728.4,7364610.4,-55.486,4.660,NaN,110,1,NaN
6,AS_GMR_31,NaN,397573.7,7365587.7,-55.474,4.659,NaN,110,1,NaN
7,AS_GMR_32,NaN,404380.9,7363075.3,-55.497,4.680,NaN,110,1,NaN
8,AS_GMR_35,NaN,397843.0,7357843.0,-55.560,4.668,NaN,110,1,NaN
9,AS_GMR_36,NaN,394408.9,7357221.6,-55.569,4.659,NaN,110,1,NaN


In [3]:
coords = df_sites[['X', 'Y']]

In [4]:
# Bring in the LCI inversion

nc_inpath = r"C:\Users\PCUser\Desktop\SSC_data\AEM_data\netcdf\SouthernStuart_WB_MGA53.nc"

d = netCDF4.Dataset(nc_inpath)

In [8]:
cond_point_utils = NetCDFPointUtils(d)

# Get the AEM utm coordinates

utm_wkt, aem_coords = cond_point_utils.utm_coords(cond_point_utils.xycoords)

In [15]:
# Extract the AEM conductivity using nearest neighbour
distances, indices = spatial_functions.nearest_neighbours(coords,
                                                          aem_coords,
                                                          points_required = 1,# return 10 closest points
                                                          max_distance = 10000.)

In [34]:
# Now we want to use these indices to extract the conductivity profile

conds = d['conductivity'][indices].data
res = 1./conds
depth_top = d['layer_top_depth'][indices].data

# Check all depths are the same

for i in range(depth_top.shape[1]):
    assert np.all(np.isclose(depth_top[:,i], depth_top[0,i]))


In [41]:
res.shape

(79, 30)

In [37]:
# calculate thickness

thickness = spatial_functions.depth_to_thickness(depth_top[0])

In [66]:
# Now we will write the first 22 layers out in the required format
# which is :
#
#nlayers
#res(layer 1) thickness(layer 1)
#...
#res(layer 1) thickness(layer n)
# res bottom half space

nlayers = 22

outdir =  r"E:\SSC\NMR\SNMR\resisitivity_profiles"

for i, (index, row) in enumerate(df_sites.iterrows()):
    fname = os.path.join(outdir, row["Field_ID"] + '_res_profile.txt')
    print(fname)
    with open(fname, 'w') as f:
        f.write(str(nlayers) + '\n')
        for j in range(nlayers - 1):
            f.write(str(round(res[i,j],4)) + '\t')
            f.write(str(round(thickness[j],4)) + '\n')
        f.write(str(round(res[i,j+1],4)))

E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_02_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_07_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_08_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_09_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_26_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_30_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_31_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_32_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_35_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_36_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_37_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_38_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_39_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_40_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS_GMR_41_res_profile.txt
E:\SSC\NMR\SNMR\resisitivity_profiles\AS

In [61]:
outdir = r"E:\SSC\NMR\SNMR\inversions"

for i, (index, row) in enumerate(df_sites.iterrows()):
    dirname = os.path.join(outdir, row["Field_ID"])
    os.mkdir(dirname)

In [59]:
path = r"E:\SSC\NMR\SNMR\Processed_data"
os.chdir(path)
import glob

files = [f for f in glob.glob(path + "**/*80ms*", recursive=True)]

In [60]:
files

[]